#### Notebook utilizado para criar a base de dados final do projeto.
##### Nesse notebook será:
1. Calculado os indicadores de complexidade econômica com base na massa salarial utilizando os dados da RAIS;
2. Calculada a Vantagem Comparativa Revelada com base no carteira ativa de crédito utilizando os dados do SCR;
3. Criada base de dados final com a unificação das bases da RAIS e do SCR;
4. Calculado os indicadores de convergência entre diversificação, ubiquidade e crédito.

In [1]:
# Importando bibliotecas
import pandas as pd
import json
import funcoes

#### 1 - Cálculo dos indicadores de complexidade econômica.

In [2]:
# Importando a base de dados da RAIS
rais = pd.read_pickle('../assets/rais2022.pkl', compression='gzip')
# Criando uma coluna para identificação da uf com base no código do município
rais['UF'] = rais.Município.apply(lambda x: str(x)[:2])
# Criando base com informações agrupadas por CNAE Subclasses e estado
rais_estados = rais[['ID CNAE', 'UF', 'Massa Salarial', 'Número Empregos']].groupby(['ID CNAE', 'UF'], as_index=False).sum()
# Excluindo informações com estado não identificado. Corresponde as informações do arquivo RAIS_VINC_PUB_NI.txt
rais_estados.drop(rais_estados[rais_estados.UF == '99'].index, inplace=True)
# Adicionando coluna com o período analisado
rais_estados['tempo'] = 2022

In [3]:
# Definindo as colunas que serão utilizadas para o cálculo dos indicadores
cols = {'tempo':'tempo', 'lugar':'UF', 'setor':'ID CNAE', 'valor':'Massa Salarial'}

# Calculando os indicadores com a função calcule_ic definida em funcoes.py
df_rais = funcoes.calcule_ic(rais_estados, cols)
df_rais

,tempo,lugar,setor,valor,QL,M,Kr,Ks,ICE,ICS
0,2022,11,977,38547.65,5.700212,1,120,13,-0.693886,-1.032172
1,2022,12,977,0.00,0.000000,0,71,13,-0.905322,-1.032172
2,2022,13,977,18458.57,1.409636,1,132,13,0.963567,-1.032172
3,2022,15,977,46376.16,1.990775,1,127,13,-0.865805,-1.032172
4,2022,16,977,1639.73,0.473008,0,49,13,-0.934358,-1.032172
...,...,...,...,...,...,...,...,...,...,...
14966,2022,42,99008,34696.24,0.039650,0,268,2,1.613737,0.530395
14967,2022,43,99008,367125.85,0.344437,0,248,2,1.406370,0.530395
14968,2022,50,99008,3130.00,0.012608,0,132,2,-0.559776,0.530395
14969,2022,52,99008,1432.34,0.002737,0,197,2,-0.167290,0.530395


#### 2 - Cálculo da VCR com base na carteira de crédito ativa.

In [4]:
# Importando a base de dados do SCR
scr = pd.read_pickle('../assets/scr122022.pkl', compression='gzip')
# Importando arquivo json com relação códigos do IBGE para ufs
with open("../assets/ufs.json", "r", encoding="utf-8") as arquivo:
    ibge_ufs = json.load(arquivo)
# Criando a coluna 'ID IBGE'
scr['ID IBGE'] = scr.uf.apply(lambda x: ibge_ufs['uf'][x])
# Ajustando a informação do período para o padrão 'AAAA'.
scr.data_base = scr.data_base.apply(lambda x: str(x)[:4])

In [5]:
# Definindo as colunas que serão utilizadas para o cálculo das VCR.
scr_cols = {'tempo':'data_base', 'lugar':'ID IBGE', 'setor':'ID CNAE', 'valor':'carteira_ativa'}
# Criando o DataFrame com as informações agrupadas por estado
scr_estados = scr[['data_base','ID IBGE','ID CNAE','carteira_ativa']].groupby(['data_base','ID IBGE','ID CNAE'], as_index=False).sum()
# Calculando as Vantagens Comparativas Reveladas com a função calcula_m definida em funcoes.py
df_scr = funcoes.calcule_m(scr_estados, scr_cols)
df_scr

,tempo,lugar,setor,valor,QL,M
0,2022,11,7227,2.902042e+05,236.969676,1
1,2022,11,,1.116011e+09,2.580277,1
2,2022,11,01156,2.166557e+07,0.380900,0
3,2022,11,01211,6.545621e+04,0.096559,0
4,2022,11,01512,1.185287e+08,7.117856,1
...,...,...,...,...,...,...
11733,2022,53,96017,1.059543e+07,0.750813,0
11734,2022,53,96025,7.986034e+07,1.007907,1
11735,2022,53,96033,3.749992e+06,0.172612,0
11736,2022,53,96092,1.123519e+07,0.879061,0


#### 3 - Unificação das bases de dados.

In [6]:
# Ajustando colunas em ambas as bases para compatibilidade de tipo.
df_rais.lugar = df_rais.lugar.astype('int')
df_scr.tempo = df_scr.tempo.astype('int')
# Removendo as informação que não possuem CNAE Subclasses.
# Note que elas foram utilizadas para o cálculo das VCR's.
df_scr.drop(df_scr[df_scr.setor == ''].index, inplace=True)
df_scr.setor = df_scr.setor.astype('int')

# Merge com 'inner' entre as bases de dados.
df = df_rais.merge(df_scr, how='inner', on=['tempo','lugar','setor'])
df

,tempo,lugar,setor,valor_x,QL_x,M_x,Kr,Ks,ICE,ICS,valor_y,QL_y,M_y
0,2022,15,1113,1935221.49,0.445574,0,127,11,-0.865805,-1.251625,1685119.66,0.149084,0
1,2022,17,1113,5147058.00,4.521237,1,87,11,-1.140667,-1.251625,14174343.20,3.869771,1
2,2022,21,1113,1612660.36,0.599872,0,98,11,-0.974768,-1.251625,23317.58,0.003543,0
3,2022,22,1113,3037142.86,2.130933,1,92,11,-0.954384,-1.251625,106662.28,0.019411,0
4,2022,23,1113,189086.81,0.039226,0,145,11,0.196568,-1.251625,117845.20,0.007845,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11581,2022,43,97005,204604.60,1.907823,1,248,8,1.406370,-0.931946,4942429.81,2.285154,1
11582,2022,50,97005,13970.14,0.559268,0,132,8,-0.559776,-0.931946,529363.49,1.303070,1
11583,2022,51,97005,14574.69,0.425173,0,112,8,-0.821356,-0.931946,1257346.50,1.251807,1
11584,2022,52,97005,126588.13,2.403984,1,197,8,-0.167290,-0.931946,869132.13,0.855754,0


#### 4 - Cálculo dos indicadores de convergência entre diversificação, ubiquidade e crédito.

In [7]:
# Função para criação da Matriz C, quando há VCR com base na massa salarial e no crédito.
def matrizC(x,y):
    return int((x==1) & (y==1))

# Função para criação da Matriz D, quando há apenas VCR com base na massa salarial.
def matrizD(x,y):
    return int((x==1) & (y==0))

# Criando colunas
df['C'] = df.apply(lambda row: matrizC(row.M_x, row.M_y), axis=1)
df['D'] = df.apply(lambda row: matrizD(row.M_x, row.M_y), axis=1)

# Crinado Medidas com base em C e D
df['Co_r'] = df.groupby(['tempo','lugar'])['C'].transform('sum')
df['Di_r'] = df.groupby(['tempo','lugar'])['D'].transform('sum')
df['Co_s'] = df.groupby(['tempo','setor'])['C'].transform('sum')
df['Di_s'] = df.groupby(['tempo','setor'])['D'].transform('sum')

# Calculando indicadores de convergência com a função calcule_iconv definida em funcoes.py.
# ICDC - Índice de Convergência entre Diversificação e Crédito
df['ICDC'] = df.apply(lambda row: funcoes.calcule_iconv(row.Co_r,row.Di_r), axis=1)
# ICUC - Índice de Convergência entre Ubiquidade e Crédito
df['ICUC'] = df.apply(lambda row: funcoes.calcule_iconv(row.Co_s,row.Di_s), axis=1)
df[['tempo','lugar','setor','ICE','ICDC','ICS','ICUC']]

,tempo,lugar,setor,ICE,ICDC,ICS,ICUC
0,2022,15,1113,-0.865805,0.214953,-1.251625,0.111111
1,2022,17,1113,-1.140667,0.074627,-1.251625,0.111111
2,2022,21,1113,-0.974768,0.012346,-1.251625,0.111111
3,2022,22,1113,-0.954384,0.028571,-1.251625,0.111111
4,2022,23,1113,0.196568,0.174603,-1.251625,0.111111
...,...,...,...,...,...,...,...
11581,2022,43,97005,1.406370,0.502110,-0.931946,-0.250000
11582,2022,50,97005,-0.559776,0.357798,-0.931946,-0.250000
11583,2022,51,97005,-0.821356,0.085714,-0.931946,-0.250000
11584,2022,52,97005,-0.167290,0.480663,-0.931946,-0.250000


In [8]:
# Salvando base de dados final
df.to_csv('../assets/df_2022.csv', index=False)